# [9665] Homework 3 : Word2vec
Data file:
* https://raw.githubusercontent.com/vjavaly/Baruch-CIS-9665/main/data/Shakespeare_plays.csv

## Homework Submission Rules (for all homework assignments)
* Homework is due by 6:05 PM on the due date
  * No late submission will be accepted
* Verify that you are submitting the correct homework file
* Homework file naming convention
  * LastName_FirstName_HwX.ipynb  [Replace X with the homework #]
    * 0.5 points deducted for submitting homework not complying with naming convention
* Before submission, execute "Kernel -> Restart Kernel and Run All Cells"
  * 0.5 points deducted for not submitting a cleanly executed notebook

## Homework 3 Requirements
* Load data into dataframe
* Perform text preprocessing
* Prepare data for training
* Use Gensim Phrases package to automatically detect common phrases (bigrams)
* Train Gensim Word2Vec model, including building vocabulary
* Explore trained model, answer the following questions:  
  1) Ask the model to find the words most similar to some of the most iconic characters from Shakespeare's plays:  
     - 'hamlet'
     - 'macbeth'

  2) Ask the model to rate how similar are 2 words:
    - 'cleopatra', 'mark_antony'
    - 'macbeth', 'hamlet'
  
  3) Ask the model to display the word that does not belong to the list:
    - ['goneril', 'edmund', 'regan']
    - ['falstaff', 'hal', 'henry_iv']
  
  4) Ask the model:
    - Which word is to 'macbeth' as 'othello' is to 'iago'?
    - Which word is to 'king_lear' as 'polonius' is to 'laertes'?


In [ ]:
from datetime import datetime
print(f'Run time: {datetime.now().strftime("%D %T")}')

Run time: 04/23/25 23:30:53


### Import libraries

In [ ]:
%%time

! pip install gensim

CPU times: user 56.6 ms, sys: 12.2 ms, total: 68.8 ms
Wall time: 7.47 s


In [ ]:
!pip install pyldavis

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade gensim

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using 

In [ ]:
import pandas as pd
import re
import string
import logging
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import multiprocessing
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load data

In [ ]:
# Read data file Shakespeare_plays.csv
df=pd.read_csv('https://raw.githubusercontent.com/vjavaly/Baruch-CIS-9665/main/data/Shakespeare_plays.csv')

### Examine data

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


### Prepare data

In [ ]:
# Review columns and drop unnecessary columns
df.drop(['Dataline','Play','PlayerLinenumber','ActSceneLine'], axis=1, inplace=True)
df.head()

,Player,PlayerLine
0,NaN,ACT I
1,NaN,SCENE I. London. The palace.
2,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
3,KING HENRY IV,"So shaken as we are, so wan with care,"
4,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [ ]:
# Check for missing values
df.isnull().sum()

,0
Player,7
PlayerLine,0


In [ ]:
# Drop rows with missing values
df=df.dropna().reset_index(drop=True)
df.isnull().sum()

,0
Player,0
PlayerLine,0


In [ ]:
# Combine relevant columns for Word2Vec model training
df['combined_text']=df['Player']+' '+df['PlayerLine']

In [ ]:
df['combined_text']

,combined_text
0,"KING HENRY IV So shaken as we are, so wan with care,"
1,"KING HENRY IV Find we a time for frighted peace to pant,"
2,KING HENRY IV And breathe short-winded accents of new broils
3,KING HENRY IV To be commenced in strands afar remote.
4,KING HENRY IV No more the thirsty entrance of this soil
...,...
111384,"LEONTES Lead us from hence, where we may leisurely"
111385,LEONTES Each one demand an answer to his part
111386,LEONTES Perform'd in this wide gap of time since first
111387,LEONTES We were dissever'd: hastily lead away.


### Perform necessary text preprocessing

In [ ]:
lem = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punct = string.punctuation

In [ ]:
# Create function to clean_text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).translate(str.maketrans('', '', punct)).lower()
    tokens = text.split()
    tokens = [lem.lemmatize(word) for word in tokens if word not in stop]
    if len(tokens) > 2:
        return ' '.join(tokens)

In [ ]:
%%time

# Clean combined text column
df['cleaned_combined_text']=df['combined_text'].apply(clean_text)

CPU times: user 9.85 s, sys: 193 ms, total: 10 s
Wall time: 15.5 s


In [ ]:
# Display the first few rows of cleaned combined text column
df['cleaned_combined_text'].head(15)

,cleaned_combined_text
0,king henry iv shaken wan care
1,king henry iv find time frighted peace pant
2,king henry iv breathe shortwinded accent new broil
3,king henry iv commenced strand afar remote
4,king henry iv thirsty entrance soil
5,king henry iv shall daub lip childrens blood
6,king henry iv shall trenching war channel field
7,king henry iv bruise floweret armed hoof
8,king henry iv hostile pace opposed eye
9,king henry iv like meteor troubled heaven


In [ ]:
# Drop rows with missing values
df = df.dropna().reset_index(drop=True)
df.shape

(105143, 4)

In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.shape

(104660, 4)

In [ ]:
# Tokenize cleaned combined text column
df['cleaned_combined_text']=df['cleaned_combined_text'].apply(lambda x: x.split())

In [ ]:
# Display first few rows after tokenization
df.head()

,Player,PlayerLine,combined_text,cleaned_combined_text
0,KING HENRY IV,"So shaken as we are, so wan with care,","KING HENRY IV So shaken as we are, so wan with care,","[king, henry, iv, shaken, wan, care]"
1,KING HENRY IV,"Find we a time for frighted peace to pant,","KING HENRY IV Find we a time for frighted peace to pant,","[king, henry, iv, find, time, frighted, peace, pant]"
2,KING HENRY IV,And breathe short-winded accents of new broils,KING HENRY IV And breathe short-winded accents of new broils,"[king, henry, iv, breathe, shortwinded, accent, new, broil]"
3,KING HENRY IV,To be commenced in strands afar remote.,KING HENRY IV To be commenced in strands afar remote.,"[king, henry, iv, commenced, strand, afar, remote]"
4,KING HENRY IV,No more the thirsty entrance of this soil,KING HENRY IV No more the thirsty entrance of this soil,"[king, henry, iv, thirsty, entrance, soil]"


### Use Gensim Phrases package to automatically detect common phrases (bigrams)

In [ ]:
# Configure logging to get output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [ ]:
# Create the relevant phrases from list of sentences
phrases = Phrases(df['cleaned_combined_text'], min_count=30, progress_per=10000)

In [ ]:
# Use Phraser() to cut down memory consumption of Phrases(), by discarding model states not strictly needed for the bigram detection task
bigram = Phraser(phrases)

In [ ]:
# Transform the corpus based on the bigrams detected
sentences = bigram[df['cleaned_combined_text']]

### Train Gensim Word2Vec model

In [ ]:
# Count the number of cores in a computer and use this info when training Word2Vec model
cores = multiprocessing.cpu_count()
cores

2

In [ ]:
# Instantiate Word2Vec model
w2v_model = Word2Vec(min_count=30,
                     window=2,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=cores-1)

In [ ]:
%%time

# Build the vocabulary table (digest the words, filter out the unique words, and do some basic counts on them)
w2v_model.build_vocab(sentences, progress_per=10000)

CPU times: user 628 ms, sys: 5.33 ms, total: 633 ms
Wall time: 632 ms


In [ ]:
%%time

# Train the model
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

CPU times: user 1min 24s, sys: 305 ms, total: 1min 24s
Wall time: 1min 2s


(5655217, 15527370)

### Explore the model

#### Most similar to

In [ ]:
# Find the words most similar to some of the most iconic characters from Shakespeare's plays: 'hamlet'
w2v_model.wv.most_similar("hamlet")

[('king_claudius', 0.89310622215271),
 ('laertes', 0.8623659014701843),
 ('lord_polonius', 0.8397166132926941),
 ('othello', 0.8336198329925537),
 ('horatio', 0.8243570327758789),
 ('queen_gertrude', 0.8210024833679199),
 ('paulina', 0.8085045218467712),
 ('leontes', 0.7969347834587097),
 ('king_lear', 0.772221028804779),
 ('sebastian', 0.7704293727874756)]

In [ ]:
# Find the words most similar to some of the most iconic characters from Shakespeare's plays: 'macbeth'
w2v_model.wv.most_similar("macbeth")

[('lady_macbeth', 0.8726910948753357),
 ('banquo', 0.8668791651725769),
 ('macduff', 0.8541286587715149),
 ('malcolm', 0.8116740584373474),
 ('duncan', 0.801443874835968),
 ('ross', 0.7907202839851379),
 ('imogen', 0.7386460304260254),
 ('cymbeline', 0.7370874285697937),
 ('posthumus_leonatus', 0.7119254469871521),
 ('edgar', 0.7086848020553589)]

#### Similarity

In [ ]:
# See how similar two words are to each other: 'cleopatra' and 'mark_antony'
w2v_model.wv.similarity("cleopatra", 'mark_antony')

0.8787465

In [ ]:
# See how similar two words are to each other: 'macbeth' and 'hamlet'
w2v_model.wv.similarity("macbeth", 'hamlet')

0.59394115

#### Does not belong (odd-one-out)

In [ ]:
# Display the word that does not belong to the list: ['goneril', 'edmund', 'regan']
w2v_model.wv.doesnt_match(['goneril', 'edmund', 'regan'])

'edmund'

In [ ]:
# Display the word that does not belong to the list: ['falstaff', 'hal', 'henry_iv']
w2v_model.wv.doesnt_match(["falstaff", "hal", "henry_iv"])

'falstaff'

#### Analogy differences

In [ ]:
# Which word is to 'macbeth' as 'othello' is to 'iago'?
w2v_model.wv.most_similar(positive=["macbeth", "othello"], negative=["iago"], topn=5)

[('macduff', 0.8136084675788879),
 ('banquo', 0.801937997341156),
 ('lady_macbeth', 0.7837125062942505),
 ('queen_margaret', 0.7641648054122925),
 ('ross', 0.7632091641426086)]

In [ ]:
# Which word is to 'king_lear' as 'polonius' is to 'laertes'?
w2v_model.wv.most_similar(positive=["king_lear", "lord_polonius"], negative=["laertes"], topn=5)

[('first_gaoler', 0.7635818123817444),
 ('kent', 0.7522541880607605),
 ('olivia', 0.7432266473770142),
 ('launce', 0.740451455116272),
 ('cloten', 0.7374950647354126)]